In [ ]:
source(paste0(dirname(getwd()),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
library( readxl )

In [ ]:
patients <- list.files(ISOFOX_DIR)

#### 0 - Collected and process adjusted TPMs from files

In [ ]:
iso <- fread(paste0(TMP_DIR, "isofox_adj_tmp.csv"))

In [ ]:
iso_base <- log(data.frame(t(iso %>% select(-GeneId) %>% column_to_rownames("GeneName"))) + 1)

#### 1 - Add in Gene Sets

In [ ]:
gene_sets <- readRDS(paste0(REF_DIR, 'gene_sets.Rds'))
gene_sets <- c(gene_sets[grep("KEGG|HALLMARK", names(gene_sets))])

In [ ]:
computer <- function( i, df ) {
  tmp <- data.frame( apply(df %>% select(any_of(gene_sets[[i]])),1,mean) )
  colnames(tmp) <- i
  tmp %>% rownames_to_column("sampleId")
}

In [ ]:
computed_sets <- list()
system.time(
for( i in names(gene_sets)){ 
  computed_sets[[i]] <- computer(i, iso_base)
})

In [ ]:
gene_sets_base <- computed_sets %>% reduce(inner_join, by = "sampleId")

#### 2 - Send the files

- Isofox gene expression

In [ ]:
isofox_ready <- iso_base
colnames(isofox_ready) <- paste0("rna_", colnames(iso_base))
isofox_ready <- isofox_ready %>% rownames_to_column("sampleId")

In [ ]:
fwrite( isofox_ready, paste0(READY_DIR, "isofox_genes_ready.csv") )

- Add gene sets

In [ ]:
gene_sets_ready <- gene_sets_base
colnames(gene_sets_ready) <- c("sampleId", paste0("rna_geneset_", colnames(gene_sets_base)[-1]))

In [ ]:
fwrite( gene_sets_ready, paste0(READY_DIR, "isofox_genesets_ready.csv"))